## 前処理用のコード

- jukyoは使わない
- 地積の公募値と実測値が異なるかを示すフラグを入れる
- 用途が二つある場合があるので、建ぺい率の大きい方を採用し、yoto, kempei, yosekiの各列にする。

In [2]:
import numpy as np
import pandas as pd
import math

In [3]:
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')

In [20]:
genba = preprocess(train_genba)

In [21]:
genba.to_csv('data/processed_train_genba.csv', index=False)

In [19]:
def preprocess(genba):
    df = genba.copy()
    
    # jukyoは使用しないのでを削除
    df = df.drop('jukyo',axis=1)
    
    # 地積の公募値と実測値の差を特徴量に追加
    df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']
    
    # 二つの用途から、一つの用途、建ぺい率、容積率を返却する
    # yotoはダミー変数化する
    df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)
    df = pd.merge(df, pd.get_dummies(df[['yoto']],drop_first = False), left_index = True, right_index = True)
    
    # 上水種別を数値化
    df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)
    
    # 下水種別を数値化
    df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)
    
    # ガス種別を数値化
    df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)
    
    # 雨水（usui）をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)
    
    # 欠損値処理
    # 建て売り数 0で補完（trainデータのID:726は、一件だけの販売なので、他の建て売り数は０で正しい
    df = df.fillna({'tateuri_su': 0, 'tochiuri_su':0 , 'joken_su':0 })
    
    # 同現場での階数ごとの販売数は、gotoデータとつきあわせると正確に出そうだが、とりあえず０で補完する。
    df = df.fillna({'hy1f_date_su':0, 'hy2f_date_su':0, 'hy3f_date_su':0})
    
    # 道路
    # 扱いが難しい。road1〜road4まであるが、road1だけのものもあれば、road1〜4まで４つあるものまで。
    # とりあえず、road1だけを採用し、接する道路数を表す項目を追加することにする。
    # Future TODO
    # road2〜road4のデータの採用のしかた
    
    # road1
    # road1_hkとroad1_sbをダミー変数化する。
    # road2〜road4の有無で、道路数を表すroad_numを作る
    # TODO: road2〜road4を削除
    df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)
    df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)
    
    #kaoku_umとyheki_umuを0と1で置き換え
    df['kaoku_um']=df.apply(lambda x: 0 if x['kaoku_um']=="（無）" else 1, axis=1)
    df['yheki_umu']=df.apply(lambda x: 0 if x['yheki_umu']=="（無）" else 1, axis=1)
    
    #yheki_yohiを0と1で置き換え
    df['yheki_yohi']=df.apply(lambda x: 0 if x['yheki_yohi']=="（不要）" else 1, axis=1)
    
    #kborjsを0と1で置き換え。"公募"に変換ミスがあって、３通りあるため、「実測」で値を決める
    df['kborjs']=df.apply(lambda x: 0 if x['kborjs']=='実測' else 1, axis=1)
    
    # 引渡の状態 hw_status
    # 善し悪しを表しそうなので数値化したいが、とりあえずはダミー変数化する。
    df = pd.merge(df, pd.get_dummies(df[['hw_status']],drop_first = False), left_index = True, right_index = True)
    
    #都市計画区域別 toshikuiki1とtoshikuiki２　toshikuiki2があればそちらを採用する。
    #一度toshikuiki2がカラの場合にtoshikuiki(新設)に特殊な値を入れて比較しないとうまくいかない。他に方法がありそうだが
    df['toshikuiki']=df['toshikuiki2'].fillna('empty')
    df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)
    df = pd.merge(df, pd.get_dummies(df[['toshikuiki']],drop_first = False), left_index = True, right_index = True)
    
    # 高度地区(kodochiku) ダミー変数化する
    df = pd.merge(df, pd.get_dummies(df[['kodochiku']],drop_first = False), left_index = True, right_index = True)
    
    # chikukeikaku, keikakuroad, kaihatsukyoka, t53kyoka, hokakyokaを0,1に変換
    df['chikukeikaku']=df.apply(lambda x: 0 if x['chikukeikaku']=="（無）" else 1, axis=1)
    df['keikakuroad']=df.apply(lambda x: 0 if x['keikakuroad']=="（無）" else 1, axis=1)
    df['kaihatsukyoka']=df.apply(lambda x: 0 if x['kaihatsukyoka']=="（不要）" else 1, axis=1)
    df['t53kyoka']=df.apply(lambda x: 0 if x['t53kyoka']=="（不要）" else 1, axis=1)
    df['hokakyoka']=df.apply(lambda x: 0 if x['hokakyoka']=="（不要）" else 1, axis=1)
    
    # 防火地区をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['bokachiiki']],drop_first = False), left_index = True, right_index = True)
    
    # 最低敷地面積(minmenseki) 欠損がある。ゼロで補完する
    df['minmenseki'] = df['minmenseki'].fillna(0)
    
    # その他規制①～④(hokakisei1～4)
    # 四つの列でダミー変数化する必要あり。
    # TODO :  hokakisei1～3のダミー変数化
    
    # kinshijikoを0,1に変換
    df['kinshijiko']=df.apply(lambda x: 0 if x['kinshijiko']=="（無）" else 1, axis=1)
    
    # 路線価格(rosenka_hb)
    # 公示価格(koji_hb)
    # 基準地価(kijun_hb)
    # 路線価格と基準地価には欠損(値が0)あり。公示価格は欠損なし。
    # 公示価格だけを残す。
    df = df.drop(['rosenka_hb','kijun_hb'],axis=1)

    # 有効宅地面積(mseki_yt_hb)
    # 道路(mseki_rd_hb)
    # ゴミ置場・公園等(mseki_dp_hb)
    # 相関不明。そのまま残しておく。
    
    # 土地最小面積～建物平均面積
    # tc_mseki_min_hb～tt_mseki_avg_hb
    # 地下に関係ないと思われるので落とす。
    df = df.drop(['tc_mseki_min_hb','tc_mseki_max_hb','tt_mseki_min_hb','tt_mseki_max_hb','tc_mseki_avg_hb','tt_mseki_avg_hb'], axis=1)
    
    # 幅員4m未満道路の通過(fi4m_yohi)と幅員3m以上4m未満道路の通過距離(fi4m_kyori)
    # fi4m_yoshiが否なら、fi4m_kyoriが０。fi3m...の項目も同様。
    # よって、fi4m_yohiとfi3m_yohiは削除する。
    # かつ、fi4m_kyori, fi3m_kyoriが欠損しているところは、yohiがちゃんと否になっているので、0で埋める。
    df = df.drop(['fi4m_yohi','fi3m_yohi'],axis=1)
    df['fi4m_kyori']=df['fi4m_kyori'].fillna(0)
    df['fi3m_kyori']=df['fi3m_kyori'].fillna(0)
    
    # バス要否、バス本数
    # 後続の交通手段のところで、バスを使うことになっていないにもかかわらず、要否が要になっているデータがあり
    # 信ぴょう性に欠けるので、削除する。
    df = df.drop(['bus_yohi','bus_hon'], axis=1)
    
    # コンビニ有無(sho_conv)～騒音有無(shu_soon)
    # ２値データ。ありは「〇」。これを0/1に変換する。
    df['sho_conv'] = df['sho_conv'].map({'○':1, np.NaN:0})
    df['sho_super'] = df['sho_super'].map({'○':1, np.NaN:0})
    df['sho_shoten'] = df['sho_shoten'].map({'○':1, np.NaN:0})    
    df['sho_market'] = df['sho_market'].map({'○':1, np.NaN:0})
    df['shu_jutaku'] = df['shu_jutaku'].map({'○':1, np.NaN:0})
    df['shu_park'] = df['shu_park'].map({'○':1, np.NaN:0})
    df['shu_shop'] = df['shu_shop'].map({'○':1, np.NaN:0})
    df['shu_factory'] = df['shu_factory'].map({'○':1, np.NaN:0})
    df['shu_hvline'] = df['shu_hvline'].map({'○':1, np.NaN:0})
    df['shu_tower'] = df['shu_tower'].map({'○':1, np.NaN:0})
    df['shu_bochi'] = df['shu_bochi'].map({'○':1, np.NaN:0})
    df['shu_sogi'] = df['shu_sogi'].map({'○':1, np.NaN:0})
    df['shu_zoki'] = df['shu_zoki'].map({'○':1, np.NaN:0})
    df['shu_kokyo'] = df['shu_kokyo'].map({'○':1, np.NaN:0})
    df['shu_highway'] = df['shu_highway'].map({'○':1, np.NaN:0})
    df['shu_kaido'] = df['shu_kaido'].map({'○':1, np.NaN:0})
    df['shu_line_ari'] = df['shu_line_ari'].map({'○':1, np.NaN:0})
    df['shu_line_nashi'] = df['shu_line_nashi'].map({'○':1, np.NaN:0})
    df['shu_soon'] = df['shu_soon'].map({'○':1, np.NaN:0})
    
    # 幼稚園徒歩/小学校徒歩/中学校徒歩(gk_yoc_tm/gk_sho_tm/gk_chu_tm)
    # 欠損もなく異常値もないため、そのままにする。
    
    # 隣接物件東戸建（2F以下）～隣接物件北田畑
    # ２値データ。ありは「〇」。これを0/1に変換する
    df['rs_e_kdate2'] = df['rs_e_kdate2'].map({'○':1, np.NaN:0})
    df['rs_e_kdate3'] = df['rs_e_kdate3'].map({'○':1, np.NaN:0})
    df['rs_e_parking'] = df['rs_e_parking'].map({'○':1, np.NaN:0})
    df['rs_e_zoki'] = df['rs_e_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_e_m_ari'] = df['rs_e_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_e_m_nashi'] = df['rs_e_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_e_tahata'] = df['rs_e_tahata'].map({'○':1, np.NaN:0}) 

    df['rs_w_kdate2'] = df['rs_w_kdate2'].map({'○':1, np.NaN:0})
    df['rs_w_kdate3'] = df['rs_w_kdate3'].map({'○':1, np.NaN:0})
    df['rs_w_parking'] = df['rs_w_parking'].map({'○':1, np.NaN:0})
    df['rs_w_zoki'] = df['rs_w_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_w_m_ari'] = df['rs_w_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_w_m_nashi'] = df['rs_w_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_w_tahata'] = df['rs_w_tahata'].map({'○':1, np.NaN:0}) 
    
    df['rs_s_kdate2'] = df['rs_s_kdate2'].map({'○':1, np.NaN:0})
    df['rs_s_kdate3'] = df['rs_s_kdate3'].map({'○':1, np.NaN:0})
    df['rs_s_parking'] = df['rs_s_parking'].map({'○':1, np.NaN:0})
    df['rs_s_zoki'] = df['rs_s_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_s_m_ari'] = df['rs_s_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_s_m_nashi'] = df['rs_s_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_s_tahata'] = df['rs_s_tahata'].map({'○':1, np.NaN:0}) 
    
    df['rs_n_kdate2'] = df['rs_n_kdate2'].map({'○':1, np.NaN:0})
    df['rs_n_kdate3'] = df['rs_n_kdate3'].map({'○':1, np.NaN:0})
    df['rs_n_parking'] = df['rs_n_parking'].map({'○':1, np.NaN:0})
    df['rs_n_zoki'] = df['rs_n_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_n_m_ari'] = df['rs_n_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_n_m_nashi'] = df['rs_n_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_n_tahata'] = df['rs_n_tahata'].map({'○':1, np.NaN:0}) 

    # 路線(rosen_nm1/rosen_nm2)、駅(eki_nm1/eki_nm2)
    # 残してある公示価格に反映されていると考えて、ここでは削除する
    df[['bus', 'walk']] = df.apply(calculateLogistics, axis=1)
    df=df.drop(['rosen_nm1','eki_nm1','bas_toho1','eki_kyori1','bastei_nm1','teiho1','rosen_nm2','eki_nm2','bas_toho2','eki_kyori2','bastei_nm2','teiho2'],axis=1)

    # 最後に不要になった列を削除する
    df=df.drop(['yoto1','kempei1','yoseki1','yoto2','kempei2','yoseki2','road1_hk','road1_sb','toshikuiki1','yoto'],axis=1)
    df=df.drop(['usui','road2_hk','road2_sb','road2_fi','road2_mg','road3_hk','road3_sb','road3_fi','road3_mg','road4_hk','road4_sb','road4_fi','road4_mg'],axis=1)
    df=df.drop(['hw_status','toshikuiki','toshikuiki2','toshikuiki','kodochiku','bokachiiki'],axis=1)
        
    # 別途ダミー変数化する
    df=df.drop(['hokakisei1','hokakisei2','hokakisei3','hokakisei4'],axis=1)

    return df

### 様々な関数

#### data_definition.txtによると、用途２がある場合には用途２を使う、とあった。
#### 当初は、建ぺい率の大きい方のyotoを採用し、yoto, kempei, yosekiからなるSeriesを返却する　とした。

In [5]:
def select_yoto(x):
    # yoto2はNaNを含む
    # 値がない場合、typeはfloatで、データがある場合にはstrになる様子。
    # そこで、型がstrかどうかで、値の有無を判定する。
    if ( type(x['yoto2']) == str ):
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    else:
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1'] 
    """"
    if ( x['yoto2']==np.nan):
        yoto =x['yoto1']
        kenpei =x['kempek1']
        yoseki =x['yoseki1']
    elif ( x['kempei1']<x['kempei2']):
        yoto =x['yoto1']
        kempei = x['kempei1']
        yoseki =x['yoseki1']
    else:
        yoto = x['yoto']=x['yoto1']
        kempei = x['kempei']=x['kempei1']
        yoseki =x['yoseki1']
    """
    return pd.Series([yoto, kempei, yoseki])

In [6]:
def calculateLogistics(x):
    if ( x['bas_toho1'] == 'バス' ):
        bus = x['eki_kyori1']
        walk = x['teiho1']
    else:
        bus = 0
        walk = x['eki_kyori1']
    
    return pd.Series([bus, walk])